In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Overview

In this notebook, we will explore **two approaches to tokenization** for Large Language Models:

1. **Manual Tokenization**  
   - We will implement a custom tokenization process using **regular expressions (regex)**.  
   - This approach will split text into tokens such that:  
     - Each **word** and **special character** (`, . : ; ? _ ! " ( ) ' --`) is treated as an individual token.  
     - **Whitespace** will be used for splitting but will **not appear as tokens**.  
   - This helps us understand the basic mechanics of tokenization.

2. **Byte Pair Encoding (BPE) Tokenization**  
   - We will use a library-based implementation of **BPE tokenization**, which is widely used in LLMs like GPT.  
   - BPE combines frequent character pairs into tokens to reduce the overall vocabulary size while preserving common patterns.  
   - This approach is **more efficient** and **closer to what real-world models use**.


### 1. **Manual Tokenization**

In [2]:
with open('/content/drive/My Drive/LLM/Nebula by ChatGPT.txt', 'r', encoding='utf-8') as f:
  raw_text = f.read()

print(len(raw_text))
print(raw_text[:99])

8716
Nebula: The Cosmic Cradles of Creation
Preface
When you look at the night sky, you see thousands of


Split the given text into tokens based on the following rules:

- **Split words and the following special characters as individual tokens:**  
  `, . : ; ? _ ! " ( ) ' --`  
- **Split on whitespace, but do not include whitespace as tokens**  
- **Remove any empty strings after splitting**  

In [3]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['Nebula', ':', 'The', 'Cosmic', 'Cradles', 'of', 'Creation', 'Preface', 'When', 'you', 'look', 'at', 'the', 'night', 'sky', ',', 'you', 'see', 'thousands', 'of', 'stars', 'scattered', 'like', 'glitter', 'across', 'a', 'black', 'canvas', '.', 'It’s']


In [4]:
print(len(preprocessed))

1662


In [5]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

663


In [6]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('&', 1)
('(', 2)
(')', 3)
(',', 4)
('.', 5)
('//hubblesite', 6)
('//www', 7)
('000', 8)
('1', 9)
('10', 10)
('10%', 11)
('1054', 12)
('17th', 13)
('18th', 14)
('2', 15)
('20th', 16)
('3', 17)
('300', 18)
('4', 19)
('5', 20)
('6', 21)
('7', 22)
('8', 23)
('9', 24)
('90%', 25)
('964', 26)
(':', 27)
(';', 28)
('?', 29)
('A', 30)
('AD', 31)
('AI', 32)
('AI-driven', 33)
('Abd', 34)
('Anatomy', 35)
('Andromeda', 36)
('Andromeda—what', 37)
('Are', 38)
('Aristotle', 39)
('Art', 40)
('As', 41)
('Astronomers', 42)
('At', 43)
('Back', 44)
('Because', 45)
('Beyond', 46)
('Birth', 47)
('But', 48)
('Carina', 49)
('Carinae', 50)


In [7]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [8]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = SimpleTokenizer(vocab)
text = 'Nebulae are vast clouds of gas and dust in space, acting as the birthplaces and graveyards of stars, shaping the life cycle of the cosmos. They come in stunning forms—glowing, dark, or shattered remnants—each telling a story of creation, destruction, and renewal in the universe.'

In [10]:
text

'Nebulae are vast clouds of gas and dust in space, acting as the birthplaces and graveyards of stars, shaping the life cycle of the cosmos. They come in stunning forms—glowing, dark, or shattered remnants—each telling a story of creation, destruction, and renewal in the universe.'

In [11]:
tokenizer.encode(text)

[126,
 207,
 632,
 273,
 486,
 365,
 203,
 314,
 400,
 570,
 4,
 191,
 211,
 602,
 664,
 203,
 664,
 486,
 580,
 4,
 664,
 602,
 428,
 664,
 486,
 602,
 664,
 5,
 165,
 283,
 400,
 588,
 664,
 4,
 299,
 4,
 493,
 664,
 664,
 664,
 187,
 584,
 486,
 296,
 4,
 307,
 4,
 203,
 664,
 400,
 602,
 628,
 5]

In [12]:
tokenizer.decode(tokenizer.encode(text))

'Nebulae are vast clouds of gas and dust in space, acting as the <|unk|> and <|unk|> of stars, <|unk|> the life <|unk|> of the <|unk|>. They come in stunning <|unk|>, dark, or <|unk|> <|unk|> <|unk|> a story of creation, destruction, and <|unk|> in the universe.'

BPE

In [13]:
# !pip3 install tiktoken

In [14]:
import importlib
import tiktoken

In [15]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
text = (
    'Nebulae are enormous clouds of gas and dust scattered across the universe, serving as the cradles where stars are born. These majestic formations can span hundreds of light-years and display breathtaking colors when illuminated by young, hot stars. <|endoftext|> Some nebulae glow brightly as emission regions, while others appear dark, blocking the starlight behind them. They play a crucial role in recycling elements, giving rise to new stars and planetary systems over billions of years. Studying nebulae helps scientists understand the origin of stars, planets, and even life itself.'
)

In [18]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[45, 1765, 377, 3609, 389, 9812, 15114, 286, 3623, 290, 8977, 16830, 1973, 262, 6881, 11, 7351, 355, 262, 1067, 324, 829, 810, 5788, 389, 4642, 13, 2312, 45308, 30648, 460, 11506, 5179, 286, 1657, 12, 19002, 290, 3359, 35589, 7577, 618, 35162, 416, 1862, 11, 3024, 5788, 13, 220, 50256, 2773, 497, 15065, 3609, 19634, 35254, 355, 25592, 7652, 11, 981, 1854, 1656, 3223, 11, 12013, 262, 3491, 2971, 2157, 606, 13, 1119, 711, 257, 8780, 2597, 287, 25914, 4847, 11, 3501, 4485, 284, 649, 5788, 290, 27047, 3341, 625, 13188, 286, 812, 13, 3604, 1112, 497, 15065, 3609, 5419, 5519, 1833, 262, 8159, 286, 5788, 11, 14705, 11, 290, 772, 1204, 2346, 13]


In [19]:
tokenizer.decode(integers)

'Nebulae are enormous clouds of gas and dust scattered across the universe, serving as the cradles where stars are born. These majestic formations can span hundreds of light-years and display breathtaking colors when illuminated by young, hot stars. <|endoftext|> Some nebulae glow brightly as emission regions, while others appear dark, blocking the starlight behind them. They play a crucial role in recycling elements, giving rise to new stars and planetary systems over billions of years. Studying nebulae helps scientists understand the origin of stars, planets, and even life itself.'